In [1]:
import pandas as pd
import numpy as np
import os
import requests

import env
import wrangle as w
import explore as exp
import datetime

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# #pd.set_option('display.width', None)
# pd.set_option('display.expand_frame_repr', False)

In [3]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve


In [4]:
df = w.wrangle_coll_stage0(year_to_retrieve, app_token)

CSV file for 2022 already exists. Loading data from the CSV.


In [5]:
# df.to_csv('preprocessed_df.csv', index=False)

In [6]:
len(df)

48254

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48254 entries, 0 to 54203
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 48254 non-null  datetime64[ns]
 1   crash_date                     48254 non-null  datetime64[ns]
 2   crash_time                     48254 non-null  object        
 3   collision_id                   48254 non-null  int64         
 4   latitude                       48254 non-null  float64       
 5   longitude                      48254 non-null  float64       
 6   vehicle_type_code1             48254 non-null  object        
 7   contributing_factor_vehicle_1  48254 non-null  object        
 8   vehicle_type_code2             48254 non-null  object        
 9   contributing_factor_vehicle_2  48254 non-null  object        
 10  on_street_name                 48254 non-null  object        
 11  borough             

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48254 entries, 0 to 54203
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 48254 non-null  datetime64[ns]
 1   crash_date                     48254 non-null  datetime64[ns]
 2   crash_time                     48254 non-null  object        
 3   collision_id                   48254 non-null  int64         
 4   latitude                       48254 non-null  float64       
 5   longitude                      48254 non-null  float64       
 6   vehicle_type_code1             48254 non-null  object        
 7   contributing_factor_vehicle_1  48254 non-null  object        
 8   vehicle_type_code2             48254 non-null  object        
 9   contributing_factor_vehicle_2  48254 non-null  object        
 10  on_street_name                 48254 non-null  object        
 11  borough             

In [9]:
df.head(3)

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,on_street_name,borough,zip_code,injuries_count,deaths_count,injuries,deaths
0,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.84866,sedan,passing or lane usage improper,sedan,unsafe lane changing,grand central pkwy,queens,11368,1,0,True,False
1,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.96038,sedan,unspecified,station wagon/sport utility vehicle,unspecified,henry hudson parkway,manhattan,10027,0,0,False,False
2,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.91799,sedan,unspecified,sedan,unspecified,saint ann's avenue,bronx,10454,0,0,False,False


In [10]:
# Example usage:
info_dataframe = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
info_dataframe

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,41429,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1440,object
3,collision_id,0,0.0,0,0,48254,int64
4,latitude,0,0.0,0,0,23107,float64
5,longitude,0,0.0,0,0,19572,float64
6,vehicle_type_code1,0,0.0,0,0,189,object
7,contributing_factor_vehicle_1,0,0.0,0,0,55,object
8,vehicle_type_code2,0,0.0,0,0,270,object
9,contributing_factor_vehicle_2,0,0.0,0,0,45,object


In [11]:
# Update numerical_columns to include columns of type datetime.datetime
numerical_columns = [col for col in df.columns if isinstance(df[col][0], (np.integer, float, datetime.datetime, datetime.time, bool)) or df[col].dtype == bool]
numerical_columns

['crash_datetime',
 'crash_date',
 'crash_time',
 'collision_id',
 'latitude',
 'longitude',
 'zip_code',
 'injuries_count',
 'deaths_count',
 'injuries',
 'deaths']

In [12]:
# Lowercase the values in non-numerical columns
for column in df.columns:
    if column not in numerical_columns:
        df[column] = df[column].str.lower()

In [13]:
# Concatenate all values from 'vehicle_type_code1' and 'vehicle_type_code2' columns
all_vehicle_types = list(df['vehicle_type_code1'].values) + list(df['vehicle_type_code2'].values)

# Print the list of all vehicle types
all_vehicle_types= set(all_vehicle_types)
all_vehicle_types

{'100th truc',
 '16m',
 '2 dr sedan',
 '3 wheel sc',
 '3-door',
 '4 dr sedan',
 '50cc mini',
 '7',
 '997',
 '999',
 'a,n',
 'abulance',
 'access a r',
 'amb',
 'ambu',
 'ambulance',
 'ambulence',
 'ambulette',
 'arcimoto',
 'armored truck',
 'asphalt ro',
 'b55 excava',
 'backhoe',
 'backhoe lo',
 'beverage truck',
 'bike',
 'bobcat',
 'bobcat sno',
 'boom lift',
 'box truck',
 'boxtruck',
 'bulk agriculture',
 'bull dozer',
 'bulldozer',
 'bus',
 'cargo trai',
 'cargo van',
 'carry all',
 'cat caterp',
 'cat escaba',
 'cater',
 'charter bu',
 'chassis &',
 'chassis cab',
 'cherry pic',
 'chevrolet',
 'citibike',
 'city owned',
 'com',
 'combinatio',
 'comm truck',
 'commercial',
 'commerical',
 'con ed tru',
 'concrete mixer',
 'constructi',
 'contructio',
 'convertible',
 'crane',
 'dart bike',
 'delivery',
 'delivery t',
 'delivery v',
 'delv',
 'department',
 'detached t',
 'dirt bike',
 'dirtbike',
 'distributo',
 'dsny sanit',
 'dsny sweep',
 'dsny truck',
 'dump',
 'dump gr',
 '

In [14]:
vehicle_list = ['100th truc', '16m', '2 dr sedan', '3 wheel sc', '3-door', '4 dr sedan', '50cc mini', '7', '997', '999', 'a,n', 'abulance', 'access a r', 'amb', 'ambu', 'ambulance', 'ambulence', 'ambulette', 'arcimoto', 'armored truck', 'asphalt ro', 'b55 excava', 'backhoe', 'backhoe lo', 'beverage truck', 'bike', 'bobcat', 'bobcat sno', 'boom lift', 'box truck', 'boxtruck', 'bulk agriculture', 'bull dozer', 'bulldozer', 'bus', 'cargo trai', 'cargo van', 'carry all', 'cat caterp', 'cat escaba', 'cater', 'charter bu', 'chassis &', 'chassis cab', 'cherry pic', 'chevrolet', 'citibike', 'city owned', 'com', 'combinatio', 'comm truck', 'commercial', 'commerical', 'con ed tru', 'concrete mixer', 'constructi', 'contructio', 'convertible', 'crane', 'dart bike', 'delivery', 'delivery t', 'delivery v', 'delv', 'department', 'detached t', 'dirt bike', 'dirtbike', 'distributo', 'dsny sanit', 'dsny sweep', 'dsny truck', 'dump', 'dump gr', 'dump truck', 'dumptruck', 'e bike', 'e bike w p', 'e com', 'e scooter', 'e skateboa', 'e wheelcha', 'e- scooter', 'e-bike', 'e-scooter', 'e-skateboa', 'e-unicycle', 'e350', 'ecoline va', 'electric m', 'electric s', 'ems', 'ems bus', 'ems fdny b', 'ems truck', 'emt ambula', 'emt truck', 'enclosed body - nonremovable enclosure', 'excavator', 'f45', 'f550', 'fd apparat', 'fd engine', 'fd ladder', 'fd truck', 'fdny', 'fdny ambul', 'fdny bus', 'fdny ems', 'fdny engin', 'fdny fire', 'fdny firet', 'fdny ladde', 'fdny picku', 'fdny rig', 'fdny truck', 'fdny vehic', 'fed ex', 'fed ex tru', 'fedex box', 'fedex truc', 'fire', 'fire appar', 'fire bus', 'fire depar', 'fire engin', 'fire truck', 'fire truvk', 'firetruck', 'flat', 'flat bed', 'flat rack', 'flatbed', 'flatbed to', 'food cart', 'food deliv', 'ford', 'ford ambul', 'ford ec2', 'fork lift', 'forklift', 'freig', 'freighntli', 'freight', 'freightlin', 'frieg', 'garbage or refuse', 'garbage tr', 'gas bike', 'gas moped', 'gas scoote', 'glass rack', 'glben', 'gov vehicl', 'government', 'govt', 'hopper', 'horse carr', 'house on w', 'hyster for', 'ic', 'in line sk', 'jetski', 'johndeere', 'komatsu lo', 'left scene', 'lift', 'lift boom', 'limo', 'lmb', 'lunch wagon', 'm2', 'mac', 'mack', 'mail truck', 'maintenanc', 'mcr', 'mechanical', 'mini van', 'minibike', 'minicycle', 'minivan', 'mobility s', 'mop pad', 'mopad', 'moped', 'moped 150c', 'moped bike', 'moped clas', 'moped scoo', 'motor scoo', 'motorbike', 'motorcycle', 'motorized', 'motorized home', 'motorizeds', 'motorscoot', 'motorscooter', 'mta', 'mta bus', 'mta van', 'mta work t', 'multi-wheeled vehicle', 'n/a', 'na', 'new flyer', 'nj transit', 'nyc fdny', 'nyc fire d', 'nyc parks', 'nyc sanita', 'nyc transi', 'nyc truck', 'nycha van', 'nys ambula', 'open body', 'other', 'p psd', 'p/u', 'pallet', 'pas', 'pass', 'passenger', 'payloader', 'pedicab', 'pepsi deli', 'petit cab', 'pick up', 'pick up tr', 'pick-up', 'pick-up tr', 'pick-up truck', 'pickup', 'pickup tru', 'pickup with mounted camper', 'pk', 'postal tru', 'power shov', 'priv ambul', 'psd', 'pump', 'push cart', 'push scoot', 'ram', 'razor scoo', 'rd/s', 'red vehicl', 'refg', 'refriger t', 'refrigerated van', 'revel mope', 'rmp', 'road sweep', 'rv', 'sanitation', 'savana van', 'school bus', 'schoolbus', 'scooter', 'scooter bi', 'sedan', 'self insur', 'semi-trail', 'skateboard', 'snow plow', 'special co', 'sprinter v', 'stak', 'stake or rack', 'stand up s', 'station wagon/sport utility vehicle', 'straight t', 'street swe', 'suburban', 'suv', 'sweeper', 't350hd', 'tank', 'tanker', 'taxi', 'tk', 'tlc', 'toolcat fo', 'tow truck', 'tow truck / wrecker', 'tr', 'trac', 'tractor', 'tractor tr', 'tractor truck diesel', 'tractor truck gasoline', 'trailer', 'trailer tr', 'trailor', 'train', 'train cart', 'transit', 'transit mt', 'truck', 'truck w/tr', 'u haul', 'u haul mov', 'u-haul', 'uhaul', 'uhaul trai', 'uhaul truc', 'uhual', 'uknown', 'unk', 'unkn', 'unknown', 'ups truck', 'us postal', 'uspcs', 'usps', 'usps van', 'usps box t', 'usps mail', 'usps posta', 'usps truck', 'usps van', 'usps vehic', 'util', 'utility', 'utility tr', 'utility ve', 'utv', 'van', 'van truck', 'van/truck', 'well driller', 'wheelchair', 'white box', 'white van', 'white work', 'work truck', 'work van', 'x', 'x trailer', 'yellow bus', 'yw school']

In [15]:
vehicle_df = pd.DataFrame({'vehicle_types': vehicle_list})
vehicle_df

,vehicle_types
0,100th truc
1,16m
2,2 dr sedan
3,3 wheel sc
4,3-door
...,...
336,work van
337,x
338,x trailer
339,yellow bus


In [16]:
description_dictionary = {
    '100th truc': '100th truck',
    '16m': '16 meter vehicle',
    '2 dr sedan': '2 door sedan',
    '3 wheel sc': '3 wheel scooter',
    '3-door': '3-door vehicle',
    '4 dr sedan': '4 door sedan',
    '50cc mini': '50cc mini vehicle',
    '7': 'Vehicle type 7',
    '997': 'Vehicle type 997',
    '999': 'Vehicle type 999',
    'a,n': 'Unknown vehicle type A,N',
    'abulance': 'Ambulance',
    'access a r': 'Access vehicle',
    'amb': 'Ambulance',
    'ambu': 'Ambulance',
    'ambulance': 'Ambulance',
    'ambulence': 'Ambulance',
    'ambulette': 'Ambulette',
    'arcimoto': 'Arcimoto vehicle',
    'armored truck': 'Armored truck',
    'asphalt ro': 'Asphalt roller',
    'b55 excava': 'B55 excavator',
    'backhoe': 'Backhoe',
    'backhoe lo': 'Backhoe loader',
    'beverage truck': 'Beverage truck',
    'bike': 'Bicycle',
    'bobcat': 'Bobcat',
    'bobcat sno': 'Bobcat snowplow',
    'boom lift': 'Boom lift',
    'box truck': 'Box truck',
    'boxtruck': 'Box truck',
    'bulk agriculture': 'Bulk agriculture vehicle',
    'bull dozer': 'Bulldozer',
    'bulldozer': 'Bulldozer',
    'bus': 'Bus',
    'cargo trai': 'Cargo trailer',
    'cargo van': 'Cargo van',
    'carry all': 'Carry all vehicle',
    'cat caterp': 'Caterpillar vehicle',
    'cat escaba': 'Caterpillar escavator',
    'cater': 'Caterpillar vehicle',
    'charter bu': 'Charter bus',
    'chassis &': 'Chassis & cab',
    'chassis cab': 'Chassis cab',
    'cherry pic': 'Cherry picker',
    'chevrolet': 'Chevrolet vehicle',
    'citibike': 'CitiBike',
    'city owned': 'City-owned vehicle',
    'com': 'Commercial vehicle',
    'combinatio': 'Combination vehicle',
    'comm truck': 'Commercial truck',
    'commercial': 'Commercial vehicle',
    'commerical': 'Commercial vehicle',
    'con ed tru': 'Con Edison truck',
    'concrete mixer': 'Concrete mixer',
    'constructi': 'Construction vehicle',
    'contructio': 'Construction vehicle',
    'convertible': 'Convertible car',
    'crane': 'Crane',
    'dart bike': 'Dart bicycle',
    'delivery': 'Delivery vehicle',
    'delivery t': 'Delivery truck',
    'delivery v': 'Delivery van',
    'delv': 'Delivery vehicle',
    'department': 'Department vehicle',
    'detached t': 'Detached trailer',
    'dirt bike': 'Dirt bike',
    'dirtbike': 'Dirt bike',
    'distributo': 'Distributor vehicle',
    'dsny sanit': 'DSNY sanitation truck',
    'dsny sweep': 'DSNY sweeper',
    'dsny truck': 'DSNY truck',
    'dump': 'Dump truck',
    'dump gr': 'Dump truck (GR)',
    'dump truck': 'Dump truck',
    'dumptruck': 'Dump truck',
    'e bike': 'Electric bike',
    'e bike w p': 'Electric bike with pedal',
    'e com': 'Electric commercial vehicle',
    'e scooter': 'Electric scooter',
    'e skateboa': 'Electric skateboard',
    'e wheelcha': 'Electric wheelchair',
    'e- scooter': 'Electric scooter',
    'e-bike': 'Electric bike',
    'e-scooter': 'Electric scooter',
    'e-skateboa': 'Electric skateboard',
    'e-unicycle': 'Electric unicycle',
    'e350': 'E350 vehicle',
    'ecoline va': 'Ecoline van',
    'electric m': 'Electric moped',
    'electric s': 'Electric scooter',
    'ems': 'Emergency Medical Services vehicle',
    'ems bus': 'EMS bus',
    'ems fdny b': 'EMS FDNY bus',
    'ems truck': 'EMS truck',
    'emt ambula': 'EMT ambulance',
    'emt truck': 'EMT truck',
    'enclosed body - nonremovable enclosure': 'Enclosed body vehicle (non-removable enclosure)',
    'excavator': 'Excavator',
    'f45': 'F45 vehicle',
    'f550': 'F550 vehicle',
    'fd apparat': 'FD apparatus vehicle',
    'fd engine': 'Fire Department engine',
    'fd ladder': 'Fire Department ladder truck',
    'fd truck': 'Fire Department truck',
    'fdny': 'FDNY vehicle',
    'fdny ambul': 'FDNY ambulance',
    'fdny bus': 'FDNY bus',
    'fdny ems': 'FDNY EMS vehicle',
    'fdny engin': 'FDNY engine',
    'fdny fire': 'FDNY fire truck',
    'fdny firet': 'FDNY firetruck',
    'fdny ladde': 'FDNY ladder truck',
    'fdny picku': 'FDNY pickup truck',
    'fdny rig': 'FDNY rig vehicle',
    'fdny truck': 'FDNY truck',
    'fdny vehic': 'FDNY vehicle',
    'fed ex': 'FedEx vehicle',
    'fed ex tru': 'FedEx truck',
    'fedex box': 'FedEx box truck',
    'fedex truc': 'FedEx truck',
    'fire': 'Fire truck',
    'fire appar': 'Fire apparatus vehicle',
    'fire bus': 'Fire department bus',
    'fire depar': 'Fire department vehicle',
    'fire engin': 'Fire engine',
    'fire truck': 'Fire truck',
    'fire truvk': 'Fire truck',
    'firetruck': 'Fire truck',
    'flat': 'Flatbed truck',
    'flat bed': 'Flatbed truck',
    'flat rack': 'Flat rack vehicle',
    'flatbed': 'Flatbed truck',
    'flatbed to': 'Flatbed tow truck',
    'food cart': 'Food cart',
    'food deliv': 'Food delivery vehicle',
    'ford': 'Ford vehicle',
    'ford ambul': 'Ford ambulance',
    'ford ec2': 'Ford EC2 vehicle',
    'fork lift': 'Forklift',
    'forklift': 'Forklift',
    'freig': 'Freight truck',
    'freighntli': 'Freightliner truck',
    'freight': 'Freight truck',
    'freightlin': 'Freightliner truck',
    'frieg': 'Freight vehicle',
    'garbage or refuse': 'Garbage or refuse truck',
    'garbage tr': 'Garbage truck',
    'gas bike': 'Gasoline-powered bike',
    'gas moped': 'Gasoline-powered moped',
    'gas scoote': 'Gasoline-powered scooter',
    'glass rack': 'Glass rack vehicle',
    'glben': 'GLBEN vehicle',
    'gov vehicl': 'Government vehicle',
    'government': 'Government vehicle',
    'govt': 'Government vehicle',
    'hopper': 'Hopper vehicle',
    'horse carr': 'Horse carriage',
    'house on w': 'House on wheels',
    'hyster for': 'Hyster forklift',
    'ic': 'Unknown vehicle type',
    'in line sk': 'Unknown vehicle type',
    'jetski': 'Unknown vehicle type',
    'johndeere': 'Unknown vehicle type',
    'komatsu lo': 'Unknown vehicle type',
    'left scene': 'Unknown vehicle type',
    'lift': 'Unknown vehicle type',
    'motor scooter': 'Motor scooter',
    'motorbike': 'Motorbike',
    'mower': 'Lawn mower',
    'muni': 'Municipal vehicle',
    'munic': 'Municipal vehicle',
    'municipal': 'Municipal vehicle',
    'nyc acs va': 'NYC ACS van',
    'nyc ambula': 'NYC ambulance',
    'nyc bikesh': 'NYC bikeshare',
    'nyc bus': 'NYC bus',
    'nyc firetr': 'NYC firetruck',
    'nyc parks': 'NYC parks vehicle',
    'nyc sani': 'NYC sanitation truck',
    'nyc taxi': 'NYC taxi',
    'nyc tlc': 'NYC TLC vehicle',
    'open body': 'Open body vehicle',
    'other': 'Other type of vehicle',
    'p/sh': 'Passenger vehicle',
    'passenger': 'Passenger vehicle',
    'pedicab': 'Pedicab',
    'pick-up tr': 'Pick-up truck',
    'pickup': 'Pick-up truck',
    'police': 'Police vehicle',
    'postal tru': 'Postal truck',
    'push scoot': 'Push scooter',
    'r/v': 'Recreational vehicle',
    'recreatio': 'Recreational vehicle',
    'refg': 'Refrigerated truck',
    'refrigerat': 'Refrigerated truck',
    'rental tru': 'Rental truck',
    'road sw': 'Road sweeper',
    'rv': 'Recreational vehicle',
    'schoo': 'School bus',
    'school bus': 'School bus',
    'scoot': 'Scooter',
    'scooter': 'Scooter',
    'self': 'Self-propelled vehicle',
    'semi-truck': 'Semi-truck',
    'skateboard': 'Skateboard',
    'skid loade': 'Skid loader',
    'small com veh(4 tires)': 'Small commercial vehicle (4 tires)',
    'snow plow': 'Snow plow',
    'spc pk': 'Special purpose vehicle',
    'sport utility / station wagon': 'SUV / Station wagon',
    'station wa': 'Station wagon',
    'street cle': 'Street cleaner', 
    'lift boom': 'Lift boom vehicle',
    'limo': 'Limousine',
    'lmb': 'LMB vehicle',
    'lunch wagon': 'Lunch wagon',
    'm2': 'M2 vehicle',
    'mac': 'Mac vehicle',
    'mack': 'Mack truck',
    'mail truck': 'Mail truck',
    'maintenanc': 'Maintenance vehicle',
    'mcr': 'MCR vehicle',
    'mechanical': 'Mechanical vehicle',
    'mini van': 'Minivan',
    'minibike': 'Minibike',
    'minicycle': 'Minicycle',
    'minivan': 'Minivan',
    'mobility s': 'Mobility scooter',
    'mop pad': 'Moped',
    'mopad': 'Moped',
    'moped': 'Moped',
    'moped 150c': 'Moped 150cc',
    'moped bike': 'Moped bike',
    'moped clas': 'Moped class vehicle',
    'moped scoo': 'Moped scooter',
    'motor scoo': 'Motor scooter',
    'motorbike': 'Motorbike',
    'motorcycle': 'Motorcycle',
    'motorized': 'Motorized vehicle',
    'motorized home': 'Motorized home',
    'motorizeds': 'Motorized vehicle',
    'motorscoot': 'Motor scooter',
    'motorscooter': 'Motor scooter',
    'mta': 'MTA vehicle',
    'mta bus': 'MTA bus',
    'mta van': 'MTA van',
    'mta work t': 'MTA work truck',
    'multi-wheeled vehicle': 'Multi-wheeled vehicle',
    'n/a': 'Not available',
    'na': 'Not available',
    'new flyer': 'New Flyer vehicle',
    'nj transit': 'NJ Transit vehicle',
    'nyc fdny': 'NYC FDNY vehicle',
    'nyc fire d': 'NYC Fire Department vehicle',
    'nyc parks': 'NYC Parks Department vehicle',
    'nyc sanita': 'NYC Sanitation vehicle',
    'nyc transi': 'NYC Transit vehicle',
    'nyc truck': 'NYC truck',
    'nycha van': 'NYCHA van',
    'nys ambula': 'NYS ambulance',
    'open body': 'Open body vehicle',
    'other': 'Other type of vehicle',
    'p psd': 'PSD vehicle',
    'p/u': 'Pick-up truck',
    'pallet': 'Pallet vehicle',
    'pas': 'Passenger vehicle',
    'pass': 'Passenger vehicle',
    'passenger': 'Passenger vehicle',
    'payloader': 'Payloader vehicle',
    'pedicab': 'Pedicab',
    'pepsi deli': 'Pepsi delivery truck',
    'petit cab': 'Petit cab',
    'pick up': 'Pick-up truck',
    'pick up tr': 'Pick-up truck',
    'pick-up': 'Pick-up truck',
    'pick-up tr': 'Pick-up truck',
    'pick-up truck': 'Pick-up truck',
    'pickup': 'Pick-up truck',
    'pickup tru': 'Pick-up truck',
    'pickup with mounted camper': 'Pick-up truck with mounted camper',
    'pk': 'Pick-up truck',
    'postal tru': 'Postal truck',
    'power shov': 'Power shovel vehicle',
    'priv ambul': 'Private ambulance',
    'psd': 'PSD vehicle',
    'pump': 'Pump vehicle',
    'push cart': 'Push cart',
    'push scoot': 'Push scooter',
    'ram': 'RAM vehicle',
    'razor scoo': 'Razor scooter',
    'rd/s': 'RD/S vehicle',
    'red vehicl': 'Red vehicle',
    'refg': 'Refrigerated truck',
    'refriger t': 'Refrigerated truck',
    'refrigerated van': 'Refrigerated van',
    'revel mope': 'Revel moped',
    'rmp': 'RMP vehicle',
    'road sweep': 'Road sweeper',
    'rv': 'Recreational vehicle',
    'sanitation': 'Sanitation vehicle',
    'savana van': 'Savana van',
    'school bus': 'School bus',
    'schoolbus': 'School bus',
    'scooter': 'Scooter',
    'scooter bi': 'Scooter bike',
    'sedan': 'Sedan',
    'self insur': 'Self-insured vehicle',
    'semi-trail': 'Semi-trailer',
    'skateboard': 'Skateboard',
    'snow plow': 'Snow plow',
    'special co': 'Special commercial vehicle',
    'sprinter v': 'Sprinter van',
    'stak': 'Stak vehicle',
    'stake or rack': 'Stake or rack vehicle',
    'stand up s': 'Stand-up scooter',
    'station wagon/sport utility vehicle': 'Station wagon/SUV',
    'straight t': 'Straight truck',
    'street swe': 'Street sweeper',
    'suburban': 'Suburban vehicle',
    'suv': 'SUV',
    'sweeper': 'Sweeper vehicle',
    't350hd': 'T350HD vehicle',
    'tank': 'Tank vehicle',
    'tanker': 'Tanker truck',
    'taxi': 'Taxi',
    'tk': 'TK vehicle',
    'tlc': 'TLC vehicle',
    'toolcat fo': 'Toolcat forklift',
    'tow truck': 'Tow truck',
    'tow truck / wrecker': 'Tow truck/wrecker',
    'tr': 'TR vehicle',
    'trac': 'Tractor',
    'tractor': 'Tractor',
    'tractor tr': 'Tractor trailer',
    'tractor truck diesel': 'Diesel tractor truck',
    'tractor truck gasoline': 'Gasoline tractor truck',
    'trailer': 'Trailer',
    'trailer tr': 'Trailer truck',
    'trailor': 'Trailer',
    'train': 'Train',
    'train cart': 'Train cart',
    'transit': 'Transit vehicle',
    'transit mt': 'Transit MT vehicle',
    'truck': 'Truck',
    'truck w/tr': 'Truck with trailer',
    'u haul': 'U-Haul truck',
    'u haul mov': 'U-Haul moving truck',
    'u-haul': 'U-Haul truck',
    'uhaul': 'U-Haul truck',
    'uhaul trai': 'U-Haul trailer',
    'uhaul truc': 'U-Haul truck',
    'uhual': 'U-Haul truck',
    'uknown': 'Unknown vehicle type',
    'unk': 'Unknown vehicle type',
    'unkn': 'Unknown vehicle type',
    'unknown': 'Unknown vehicle type',
    'ups truck': 'UPS truck',
    'us postal': 'US Postal vehicle',
    'uspcs': 'USPCS vehicle',
    'usps': 'USPS vehicle',
    'usps van': 'USPS van',
    'usps box t': 'USPS box truck',
    'usps mail': 'USPS mail truck',
    'usps posta': 'USPS postal vehicle',
    'usps truck': 'USPS truck',
    'usps vehic': 'USPS vehicle',
    'util': 'Utility vehicle',
    'utility': 'Utility vehicle',
    'utility tr': 'Utility truck',
    'utility ve': 'Utility vehicle',
    'utv': 'UTV (Utility Task Vehicle)',
    'van': 'Van',
    'van truck': 'Van truck',
    'van/truck': 'Van/Truck',
    'well driller': 'Well driller vehicle',
    'wheelchair': 'Wheelchair',
    'white box': 'White box truck',
    'white van': 'White van',
    'white work': 'White work vehicle',
    'work truck': 'Work truck',
    'work van': 'Work van',
    'x': 'Unknown vehicle type',
    'x trailer': 'Unknown trailer',
    'yellow bus': 'Yellow bus',
    'yw school': 'YW School bus'}

In [17]:
# Create a new 'description' column by mapping values from 'Vehicle Names' column
vehicle_df['description'] = vehicle_df['vehicle_types'].map(description_dictionary)
vehicle_df

,vehicle_types,description
0,100th truc,100th truck
1,16m,16 meter vehicle
2,2 dr sedan,2 door sedan
3,3 wheel sc,3 wheel scooter
4,3-door,3-door vehicle
...,...,...
336,work van,Work van
337,x,Unknown vehicle type
338,x trailer,Unknown trailer
339,yellow bus,Yellow bus


In [18]:
# OLD DICTIONARY IGNORE
# cat_dictionary = {
#     '100th truck': 'Unknown',
#     '16 meter vehicle': 'Commercial Vehicles',
#     '2 door sedan': 'Personal Vehicles',
#     '3 wheel scooter': 'Two-Wheelers',
#     '3-door vehicle': 'Personal Vehicles',
#     '4 door sedan': 'Personal Vehicles',
#     '50cc mini vehicle': 'Two-Wheelers
#     'Vehicle type 7': 'Unknown',
#     'Vehicle type 997': 'Unknown',
#     'Vehicle type 999': 'Unknown',
#     'Unknown vehicle type A,N': 'Unknown',
#     'Ambulance': 'Emergency Vehicles',
#     'Access vehicle': 'Utility & Service Vehicles',
#     'Ambulette': 'Emergency Vehicles',
#     'Arcimoto vehicle': 'Special Vehicles',
#     'Armored truck': 'Special Vehicles',
#     'Asphalt roller': 'Heavy Machinery',
#     'B55 excavator': 'Heavy Machinery',
#     'Backhoe': 'Heavy Machinery',
#     'Backhoe loader': 'Heavy Machinery',
#     'Beverage truck': 'Commercial Vehicles',
#     'Bicycle': 'Two-Wheelers',
#     'Bobcat': 'Heavy Machinery',
#     'Bobcat snowplow': 'Heavy Machinery',
#     'Boom lift': 'Heavy Machinery',
#     'Box truck': 'Commercial Vehicles',
#     'Bulk agriculture vehicle': 'Commercial Vehicles',
#     'Bulldozer': 'Heavy Machinery',
#     'Bus': 'Public Transport',
#     'Cargo trailer': 'Commercial Vehicles',
#     'Cargo van': 'Commercial Vehicles',
#     'Carry all vehicle': 'Utility & Service Vehicles',
#     'Caterpillar vehicle': 'Heavy Machinery',
#     'Caterpillar escavator': 'Heavy Machinery',
#     'Charter bus': 'Public Transport',
#     'Chassis & cab': 'Commercial Vehicles',
#     'Chassis cab': 'Commercial Vehicles',
#     'Cherry picker': 'Heavy Machinery',
#     'Chevrolet vehicle': 'Personal Vehicles',
#     'CitiBike': 'Two-Wheelers',a
#     'City-owned vehicle': 'Utility & Service Vehicles',
#     'Commercial vehicle': 'Commercial Vehicles',
#     'Combination vehicle': 'Commercial Vehicles',
#     'Commercial truck': 'Commercial Vehicles',
#     'Con Edison truck': 'Utility & Service Vehicles',
#     'Concrete mixer': 'Heavy Machinery',
#     'Construction vehicle': 'Heavy Machinery',
#     'Convertible car': 'Personal Vehicles',
#     'Crane': 'Heavy Machinery',
#     'Dart bicycle': 'Two-Wheelers
#     'Delivery vehicle': 'Commercial Vehicles',
#     'Delivery truck': 'Commercial Vehicles',
#     'Delivery van': 'Commercial Vehicles',
#     'Department vehicle': 'Utility & Service Vehicles',
#     'Detached trailer': 'Commercial Vehicles',
#     'Dirt bike': 'Two-Wheelers',
#     'Distributor vehicle': 'Commercial Vehicles',
#     'DSNY sanitation truck': 'Utility & Service Vehicles',
#     'DSNY sweeper': 'Utility & Service Vehicles',
#     'DSNY truck': 'Utility & Service Vehicles',
#     'Dump truck': 'Heavy Machinery',
#     'Dump truck (GR)': 'Heavy Machinery',
#     'Electric bike': 'Two-Wheelers',
#     'Electric bike with pedal': 'Two-Wheelers
#     'Electric commercial vehicle': 'Commercial Vehicles',
#     'Electric scooter': 'Two-Wheelers
#     'Electric skateboard': 'Two-Wheelers
#     'Electric wheelchair': 'Two-Wheelers',
#     'Electric unicycle': 'Two-Wheelers',
#     'E350 vehicle': 'Commercial Vehicles',
#     'Ecoline van': 'Commercial Vehicles',
#     'Electric moped': 'Two-Wheelers
#     'Emergency Medical Services vehicle': 'Emergency Vehicles',
#     'EMS bus': 'Emergency Vehicles',
#     'EMS FDNY bus': 'Emergency Vehicles',
#     'EMS truck': 'Emergency Vehicles',
#     'EMT ambulance': 'Emergency Vehicles',
#     'EMT truck': 'Emergency Vehicles',
#     'Enclosed body vehicle (non-removable enclosure)': 'Commercial Vehicles',
#     'Excavator': 'Heavy Machinery',
#     'F45 vehicle': 'Commercial Vehicles',
#     'F550 vehicle': 'Commercial Vehicles',
#     'FD apparatus vehicle': 'Emergency Vehicles',
#     'Fire Department engine': 'Emergency Vehicles',
#     'Fire Department ladder truck': 'Emergency Vehicles',
#     'Fire Department truck': 'Emergency Vehicles',
#     'FDNY vehicle': 'Emergency Vehicles',
#     'FDNY ambulance': 'Emergency Vehicles',
#     'FDNY bus': 'Emergency Vehicles',
#     'FDNY EMS vehicle': 'Emergency Vehicles',
#     'FDNY engine': 'Emergency Vehicles',
#     'FDNY fire truck': 'Emergency Vehicles',
#     'FDNY firetruck': 'Emergency Vehicles',
#     'FDNY ladder truck': 'Emergency Vehicles',
#     'FDNY pickup truck': 'Utility & Service Vehicles',
#     'FDNY rig vehicle': 'Emergency Vehicles',
#     'FDNY truck': 'Emergency Vehicles',
#     'FedEx vehicle': 'Commercial Vehicles',
#     'FedEx truck': 'Commercial Vehicles',
#     'FedEx box truck': 'Commercial Vehicles',
#     'Fire truck': 'Emergency Vehicles',
#     'Fire apparatus vehicle': 'Emergency Vehicles',
#     'Fire department bus': 'Utility & Service Vehicles',
#     'Fire department vehicle': 'Emergency Vehicles',
#     'Fire engine': 'Emergency Vehicles',
#     'Flatbed truck': 'Commercial Vehicles',
#     'Flat rack vehicle': 'Commercial Vehicles',
#     'Flatbed tow truck': 'Utility & Service Vehicles',
#     'Food cart': 'Special Vehicles',
#     'Food delivery vehicle': 'Commercial Vehicles',
#     'Ford vehicle': 'Personal Vehicles',
#     'Ford ambulance': 'Emergency Vehicles',
#     'Ford EC2 vehicle': 'Commercial Vehicles',
#     'Forklift': 'Heavy Machinery',
#     'Freight truck': 'Commercial Vehicles',
#     'Freightliner truck': 'Commercial Vehicles',
#     'Freight vehicle': 'Commercial Vehicles',
#     'Garbage or refuse truck': 'Utility & Service Vehicles',
#     'Garbage truck': 'Utility & Service Vehicles',
#     'Gasoline-powered bike': 'Two-Wheelers', 
#     'Gasoline-powered moped': 'Two-Wheelers',
#     'Gasoline-powered scooter': 'Two-Wheelers
#     'Glass rack vehicle': 'Commercial Vehicles',
#     'GLBEN vehicle': 'Unknown',
#     'Government vehicle': 'Utility & Service Vehicles',
#     'Hopper vehicle': 'Utility & Service Vehicles',
#     'Horse carriage': 'Special Vehicles',
#     'House on wheels': 'Special Vehicles',
#     'Hyster forklift': 'Heavy Machinery',
#     'Unknown vehicle type': 'Unknown',
#     'Lift boom vehicle': 'Heavy Machinery',
#     'Limousine': 'Personal Vehicles',
#     'LMB vehicle': 'Unknown',
#     'Lunch wagon': 'Special Vehicles',
#     'M2 vehicle': 'Unknown',
#     'Mac vehicle': 'Unknown',
#     'Mack truck': 'Commercial Vehicles',
#     'Mail truck': 'Utility & Service Vehicles',
#     'Maintenance vehicle': 'Utility & Service Vehicles',
#     'MCR vehicle': 'Unknown',
#     'Mechanical vehicle': 'Unknown',
#     'Minivan': 'Personal Vehicles',
#     'Minibike': 'Two-Wheelers',
#     'Minicycle': 'Two-Wheelers
#     'Mobility scooter': 'Two-Wheelers',
#     'Moped': 'Two-Wheelers',
#     'Moped 150cc': 'Two-Wheelers
#     'Moped bike': 'Two-Wheelers
#     'Moped class vehicle': 'Two-Wheelers
#     'Moped scooter': 'Two-Wheelers',
#     'Motor scooter': 'Two-Wheelers',
#     'Motorbike': 'Two-Wheelers',
#     'Motorcycle': 'Two-Wheelers',
#     'Motorized vehicle': 'Personal Vehicles',
#     'Motorized home': 'Special Vehicles',
#     'MTA vehicle': 'Public Transport',
#     'MTA bus': 'Public Transport',
#     'MTA van': 'Public Transport',
#     'MTA work truck': 'Utility & Service Vehicles',
#     'Multi-wheeled vehicle': 'Unknown',
#     'Not available': 'Unknown',
#     'New Flyer vehicle': 'Public Transport',
#     'NJ Transit vehicle': 'Public Transport',
#     'NYC FDNY vehicle': 'Emergency Vehicles',
#     'NYC Fire Department vehicle': 'Emergency Vehicles',
#     'NYC Parks Department vehicle': 'Utility & Service Vehicles',
#     'NYC Sanitation vehicle': 'Utility & Service Vehicles',
#     'NYC Transit vehicle': 'Public Transport',
#     'NYC truck': 'Utility & Service Vehicles',
#     'NYCHA van': 'Utility & Service Vehicles',
#     'NYS ambulance': 'Emergency Vehicles',
#     'Open body vehicle': 'Commercial Vehicles',
#     'Other type of vehicle': 'Unknown',
#     'PSD vehicle': 'Unknown',
#     'Pick-up truck': 'Personal Vehicles',
#     'Pallet vehicle': 'Commercial Vehicles',
#     'Passenger vehicle': 'Personal Vehicles',
#     'Payloader vehicle': 'Heavy Machinery',
#     'Pedicab': 'Two-Wheelers',
#     'Pepsi delivery truck': 'Commercial Vehicles',
#     'Petit cab': 'Unknown',
#     'Pick-up truck with mounted camper': 'Personal Vehicles',
#     'Postal truck': 'Utility & Service Vehicles',
#     'Power shovel vehicle': 'Heavy Machinery',
#     'Private ambulance': 'Emergency Vehicles',
#     'Pump vehicle': 'Heavy Machinery',
#     'Push cart': 'Special Vehicles',
#     'Push scooter': 'Two-Wheelers
#     'RAM vehicle': 'Personal Vehicles',
#     'Razor scooter': 'Two-Wheelers',
#     'RD/S vehicle': 'Unknown',
#     'Red vehicle': 'Unknown',
#     'Refrigerated truck': 'Commercial Vehicles',
#     'Refrigerated van': 'Commercial Vehicles',
#     'Revel moped': 'Two-Wheelers
#     'RMP vehicle': 'Unknown',
#     'Road sweeper': 'Utility & Service Vehicles',
#     'Recreational vehicle': 'Special Vehicles',
#     'Sanitation vehicle': 'Utility & Service Vehicles',
#     'Savana van': 'Utility & Service Vehicles',
#     'School bus': 'Public Transport',
#     'Scooter': 'Two-Wheelers',
#     'Scooter bike': 'Two-Wheelers',
#     'Sedan': 'Personal Vehicles',
#     'Self-insured vehicle': 'Unknown',
#     'Semi-trailer': 'Commercial Vehicles',
#     'Skateboard': 'Two-Wheelers
#     'Snow plow': 'Utility & Service Vehicles',
#     'Special commercial vehicle': 'Special Vehicles',
#     'Sprinter van': 'Utility & Service Vehicles',
#     'Stak vehicle': 'Commercial Vehicles',
#     'Stake or rack vehicle': 'Commercial Vehicles',
#     'Stand-up scooter': 'Two-Wheelers
#     'Station wagon/SUV': 'Personal Vehicles',
#     'Straight truck': 'Commercial Vehicles',
#     'Street sweeper': 'Utility & Service Vehicles',
#     'Suburban vehicle': 'Personal Vehicles',
#     'SUV': 'Personal Vehicles',
#     'Sweeper vehicle': 'Utility & Service Vehicles',
#     'T350HD vehicle': 'Unknown',
#     'Tank vehicle': 'Commercial Vehicles',
#     'Tanker truck': 'Commercial Vehicles',
#     'Taxi': 'Personal Vehicles',
#     'TK vehicle': 'Unknown',
#     'TLC vehicle': 'Public Transport',
#     'Toolcat forklift': 'Heavy Machinery',
#     'Tow truck': 'Utility & Service Vehicles',
#     'Tow truck/wrecker': 'Utility & Service Vehicles',
#     'TR vehicle': 'Unknown',
#     'Tractor': 'Heavy Machinery',
#     'Tractor trailer': 'Commercial Vehicles',
#     'Diesel tractor truck': 'Commercial Vehicles',
#     'Gasoline tractor truck': 'Commercial Vehicles',
#     'Trailer': 'Commercial Vehicles',
#     'Trailer truck': 'Commercial Vehicles',
#     'Train': 'Public Transport',
#     'Train cart': 'Public Transport',
#     'Transit vehicle': 'Public Transport',
#     'Transit MT vehicle': 'Public Transport',
#     'Truck': 'Commercial Vehicles',
#     'Truck with trailer': 'Commercial Vehicles',
#     'U-Haul truck': 'Commercial Vehicles',
#     'U-Haul moving truck': 'Commercial Vehicles',
#     'U-Haul trailer': 'Commercial Vehicles',
#     'UPS truck': 'Commercial Vehicles',
#     'US Postal vehicle': 'Utility & Service Vehicles',
#     'USPCS vehicle': 'Utility & Service Vehicles',
#     'USPS vehicle': 'Utility & Service Vehicles',
#     'USPS van': 'Utility & Service Vehicles',
#     'USPS box truck': 'Utility & Service Vehicles',
#     'USPS mail truck': 'Utility & Service Vehicles',
#     'USPS postal vehicle': 'Utility & Service Vehicles',
#     'USPS truck': 'Utility & Service Vehicles',
#     'Utility vehicle': 'Utility & Service Vehicles',
#     'Utility truck': 'Utility & Service Vehicles',
#     'UTV (Utility Task Vehicle)': 'Utility & Service Vehicles',
#     'Van': 'Utility & Service Vehicles',
#     'Van truck': 'Utility & Service Vehicles',
#     'Van/Truck': 'Utility & Service Vehicles',
#     'Well driller vehicle': 'Heavy Machinery',
#     'Wheelchair': 'Personal Vehicles',
#     'White box truck': 'Commercial Vehicles',
#     'White van': 'Utility & Service Vehicles',
#     'White work vehicle': 'Utility & Service Vehicles',
#     'Work truck': 'Utility & Service Vehicles',
#     'Work van': 'Utility & Service Vehicles',
#     'Unknown trailer': 'Unknown',
#     'Yellow bus': 'Public Transport',
#     'YW School bus': 'Public Transport'
#     }

In [19]:
cat_dictionary = {
    '100th truck': 'Commercial',
    '16 meter vehicle': 'Commercial',
    '2 door sedan': 'Personal',
    '3 wheel scooter': 'Two-Wheelers',
    '3-door vehicle': 'Personal',
    '4 door sedan': 'Personal',
    '50cc mini vehicle': 'Two-Wheelers',
    'Vehicle type 7': 'Other',
    'Vehicle type 997': 'Other',
    'Vehicle type 999': 'Other',
    'Unknown vehicle type A,N': 'Other',
    'Ambulance': 'Other',
    'Access vehicle': 'Other',
    'Ambulette': 'Other',
    'Arcimoto vehicle': 'Other',
    'Armored truck': 'Other',
    'Asphalt roller': 'Other',
    'B55 excavator': 'Other',
    'Backhoe': 'Other',
    'Backhoe loader': 'Other',
    'Beverage truck': 'Commercial',
    'Bicycle': 'Two-Wheelers',
    'Bobcat': 'Other',
    'Bobcat snowplow': 'Other',
    'Boom lift': 'Other',
    'Box truck': 'Commercial',
    'Bulk agriculture vehicle': 'Commercial',
    'Bulldozer': 'Other',
    'Bus': 'Other',
    'Cargo trailer': 'Commercial',
    'Cargo van': 'Commercial',
    'Carry all vehicle': 'Commercial',
    'Caterpillar vehicle': 'Other',
    'Caterpillar escavator': 'Other',
    'Charter bus': 'Other',
    'Chassis & cab': 'Commercial',
    'Chassis cab': 'Commercial',
    'Cherry picker': 'Other',
    'Chevrolet vehicle': 'Personal',
    'CitiBike': 'Two-Wheelers',
    'City-owned vehicle': 'Other',
    'Commercial vehicle': 'Commercial',
    'Combination vehicle': 'Commercial',
    'Commercial truck': 'Commercial',
    'Con Edison truck': 'Other',
    'Concrete mixer': 'Other',
    'Construction vehicle': 'Other',
    'Convertible car': 'Personal',
    'Crane': 'Other',
    'Dart bicycle': 'Two-Wheelers',
    'Delivery vehicle': 'Commercial',
    'Delivery truck': 'Commercial',
    'Delivery van': 'Commercial',
    'Department vehicle': 'Other',
    'Detached trailer': 'Commercial',
    'Dirt bike': 'Two-Wheelers',
    'Distributor vehicle': 'Commercial',
    'DSNY sanitation truck': 'Other',
    'DSNY sweeper': 'Other',
    'DSNY truck': 'Other',
    'Dump truck': 'Other',
    'Dump truck (GR)': 'Other',
    'Electric bike': 'Two-Wheelers',
    'Electric bike with pedal': 'Two-Wheelers',
    'Electric commercial vehicle': 'Commercial',
    'Electric scooter': 'Two-Wheelers',
    'Electric skateboard': 'Two-Wheelers',
    'Electric wheelchair': 'Other',
    'Electric unicycle': 'Two-Wheelers',
    'E350 vehicle': 'Commercial',
    'Ecoline van': 'Commercial',
    'Electric moped': 'Two-Wheelers',
    'Emergency Medical Services vehicle': 'Other',
    'EMS bus': 'Other',
    'EMS FDNY bus': 'Other',
    'EMS truck': 'Other',
    'EMT ambulance': 'Other',
    'EMT truck': 'Other',
    'Enclosed body vehicle (non-removable enclosure)': 'Commercial',
    'Excavator': 'Other',
    'F45 vehicle': 'Commercial',
    'F550 vehicle': 'Commercial',
    'FD apparatus vehicle': 'Other',
    'Fire Department engine': 'Other',
    'Fire Department ladder truck': 'Other',
    'Fire Department truck': 'Other',
    'FDNY vehicle': 'Other',
    'FDNY ambulance': 'Other',
    'FDNY bus': 'Other',
    'FDNY EMS vehicle': 'Other',
    'FDNY engine': 'Other',
    'FDNY fire truck': 'Other',
    'FDNY firetruck': 'Other',
    'FDNY ladder truck': 'Other',
    'FDNY pickup truck': 'Other',
    'FDNY rig vehicle': 'Other',
    'FDNY truck': 'Other',
    'FedEx vehicle': 'Commercial',
    'FedEx truck': 'Commercial',
    'FedEx box truck': 'Commercial',
    'Fire truck': 'Other',
    'Fire apparatus vehicle': 'Other',
    'Fire department bus': 'Other',
    'Fire department vehicle': 'Other',
    'Fire engine': 'Other',
    'Flatbed truck': 'Commercial',
    'Flat rack vehicle': 'Commercial',
    'Flatbed tow truck': 'Commercial',
    'Food cart': 'Commercial',
    'Food delivery vehicle': 'Commercial',
    'Ford vehicle': 'Personal',
    'Ford ambulance': 'Other',
    'Ford EC2 vehicle': 'Commercial',
    'Forklift': 'Other',
    'Freight truck': 'Commercial',
    'Freightliner truck': 'Commercial',
    'Freight vehicle': 'Commercial',
    'Garbage or refuse truck': 'Other',
    'Garbage truck': 'Other',
    'Gasoline-powered bike': 'Two-Wheelers',
    'Gasoline-powered moped': 'Two-Wheelers',
    'Gasoline-powered scooter': 'Two-Wheelers',
    'Glass rack vehicle': 'Commercial',
    'GLBEN vehicle': 'Other',
    'Government vehicle': 'Other',
    'Hopper vehicle': 'Commercial',
    'Horse carriage': 'Other',
    'House on wheels': 'Other',
    'Hyster forklift': 'Other',
    'Unknown vehicle type': 'Other',
    'Lift boom vehicle': 'Other',
    'Limousine': 'Personal',
    'LMB vehicle': 'Other',
    'Lunch wagon': 'Commercial',
    'M2 vehicle': 'Other',
    'Mac vehicle': 'Other',
    'Mack truck': 'Commercial',
    'Mail truck': 'Other',
    'Maintenance vehicle': 'Other',
    'MCR vehicle': 'Other',
    'Mechanical vehicle': 'Other',
    'Minivan': 'Personal',
    'Minibike': 'Two-Wheelers',
    'Minicycle': 'Two-Wheelers',
    'Mobility scooter': 'Other',
    'Moped': 'Two-Wheelers',
    'Moped 150cc': 'Two-Wheelers',
    'Moped bike': 'Two-Wheelers',
    'Moped class vehicle': 'Two-Wheelers',
    'Moped scooter': 'Two-Wheelers',
    'Motor scooter': 'Two-Wheelers',
    'Motorbike': 'Two-Wheelers',
    'Motorcycle': 'Two-Wheelers',
    'Motorized vehicle': 'Personal',
    'Motorized home': 'Other',
    'MTA vehicle': 'Other',
    'MTA bus': 'Other',
    'MTA van': 'Other',
    'MTA work truck': 'Other',
    'Multi-wheeled vehicle': 'Other',
    'Not available': 'Other',
    'New Flyer vehicle': 'Other',
    'NJ Transit vehicle': 'Other',
    'NYC FDNY vehicle': 'Other',
    'NYC Fire Department vehicle': 'Other',
    'NYC Parks Department vehicle': 'Other',
    'NYC Sanitation vehicle': 'Other',
    'NYC Transit vehicle': 'Other',
    'NYC truck': 'Other',
    'NYCHA van': 'Other',
    'NYS ambulance': 'Other',
    'Open body vehicle': 'Commercial',
    'Other type of vehicle': 'Other',
    'PSD vehicle': 'Other',
    'Pick-up truck': 'Commercial',
    'Pallet vehicle': 'Commercial',
    'Passenger vehicle': 'Personal',
    'Payloader vehicle': 'Other',
    'Pedicab': 'Other',
    'Pepsi delivery truck': 'Commercial',
    'Petit cab': 'Commercial',
    'Pick-up truck with mounted camper': 'Personal',
    'Postal truck': 'Other',
    'Power shovel vehicle': 'Other',
    'Private ambulance': 'Other',
    'Pump vehicle': 'Other',
    'Push cart': 'Commercial',
    'Push scooter': 'Two-Wheelers',
    'RAM vehicle': 'Personal',
    'Razor scooter': 'Two-Wheelers',
    'RD/S vehicle': 'Other',
    'Red vehicle': 'Other',
    'Refrigerated truck': 'Commercial',
    'Refrigerated van': 'Commercial',
    'Revel moped': 'Two-Wheelers',
    'RMP vehicle': 'Other',
    'Road sweeper': 'Other',
    'Recreational vehicle': 'Personal',
    'Sanitation vehicle': 'Other',
    'Savana van': 'Commercial',
    'School bus': 'Other',
    'Scooter': 'Two-Wheelers',
    'Scooter bike': 'Two-Wheelers',
    'Sedan': 'Personal',
    'Self-insured vehicle': 'Other',
    'Semi-trailer': 'Commercial',
    'Skateboard': 'Two-Wheelers',
    'Snow plow': 'Other',
    'Special commercial vehicle': 'Other',
    'Sprinter van': 'Commercial',
    'Stak vehicle': 'Commercial',
    'Stake or rack vehicle': 'Commercial',
    'Stand-up scooter': 'Two-Wheelers',
    'Station wagon/SUV': 'Personal',
    'Straight truck': 'Commercial',
    'Street sweeper': 'Other',
    'Suburban vehicle': 'Personal',
    'SUV': 'Personal',
    'Sweeper vehicle': 'Other',
    'T350HD vehicle': 'Commercial',
    'Tank vehicle': 'Commercial',
    'Tanker truck': 'Commercial',
    'Taxi': 'Commercial',
    'TK vehicle': 'Other',
    'TLC vehicle': 'Commercial',
    'Toolcat forklift': 'Other',
    'Tow truck': 'Commercial',
    'Tow truck/wrecker': 'Commercial',
    'TR vehicle': 'Other',
    'Tractor': 'Other',
    'Tractor trailer': 'Commercial',
    'Diesel tractor truck': 'Commercial',
    'Gasoline tractor truck': 'Commercial',
    'Trailer': 'Commercial',
    'Trailer truck': 'Commercial',
    'Train': 'Other',
    'Train cart': 'Other',
    'Transit vehicle': 'Other',
    'Transit MT vehicle': 'Other',
    'Truck': 'Commercial',
    'Truck with trailer': 'Commercial',
    'U-Haul truck': 'Commercial',
    'U-Haul moving truck': 'Commercial',
    'U-Haul trailer': 'Commercial',
    'UPS truck': 'Commercial',
    'US Postal vehicle': 'Other',
    'USPCS vehicle': 'Other',
    'USPS vehicle': 'Other',
    'USPS van': 'Other',
    'USPS box truck': 'Other',
    'USPS mail truck': 'Other',
    'USPS postal vehicle': 'Other',
    'USPS truck': 'Other',
    'Utility vehicle': 'Other',
    'Utility truck': 'Other',
    'UTV (Utility Task Vehicle)': 'Other',
    'Van': 'Commercial',
    'Van truck': 'Commercial',
    'Van/Truck': 'Commercial',
    'Well driller vehicle': 'Other',
    'Wheelchair': 'Other',
    'White box truck': 'Commercial',
    'White van': 'Commercial',
    'White work vehicle': 'Commercial',
    'Work truck': 'Commercial',
    'Work van': 'Commercial',
    'Unknown trailer': 'Other',
    'Yellow bus': 'Other',
    'YW School bus': 'Other',
    'NaN': 'Other'
    }

In [20]:
# Create a new 'description' column by mapping values from 'Vehicle Names' column
vehicle_df['category'] = vehicle_df['description'].map(cat_dictionary)
vehicle_df

,vehicle_types,description,category
0,100th truc,100th truck,Commercial
1,16m,16 meter vehicle,Commercial
2,2 dr sedan,2 door sedan,Personal
3,3 wheel sc,3 wheel scooter,Two-Wheelers
4,3-door,3-door vehicle,Personal
...,...,...,...
336,work van,Work van,Commercial
337,x,Unknown vehicle type,Other
338,x trailer,Unknown trailer,Other
339,yellow bus,Yellow bus,Other


In [21]:
# vehicle_df.to_csv('vehicle_categories_map.csv')

In [22]:
# Convert all values in the DataFrame to lowercase
vehicle_df = vehicle_df.applymap(str.lower)
vehicle_df

,vehicle_types,description,category
0,100th truc,100th truck,commercial
1,16m,16 meter vehicle,commercial
2,2 dr sedan,2 door sedan,personal
3,3 wheel sc,3 wheel scooter,two-wheelers
4,3-door,3-door vehicle,personal
...,...,...,...
336,work van,work van,commercial
337,x,unknown vehicle type,other
338,x trailer,unknown trailer,other
339,yellow bus,yellow bus,other


In [23]:
# Create a dictionary mapping vehicle_types to category
type_to_cat = dict(zip(vehicle_df['vehicle_types'], vehicle_df['category']))
type_to_cat

{'100th truc': 'commercial',
 '16m': 'commercial',
 '2 dr sedan': 'personal',
 '3 wheel sc': 'two-wheelers',
 '3-door': 'personal',
 '4 dr sedan': 'personal',
 '50cc mini': 'two-wheelers',
 '7': 'other',
 '997': 'other',
 '999': 'other',
 'a,n': 'other',
 'abulance': 'other',
 'access a r': 'other',
 'amb': 'other',
 'ambu': 'other',
 'ambulance': 'other',
 'ambulence': 'other',
 'ambulette': 'other',
 'arcimoto': 'other',
 'armored truck': 'other',
 'asphalt ro': 'other',
 'b55 excava': 'other',
 'backhoe': 'other',
 'backhoe lo': 'other',
 'beverage truck': 'commercial',
 'bike': 'two-wheelers',
 'bobcat': 'other',
 'bobcat sno': 'other',
 'boom lift': 'other',
 'box truck': 'commercial',
 'boxtruck': 'commercial',
 'bulk agriculture': 'commercial',
 'bull dozer': 'other',
 'bulldozer': 'other',
 'bus': 'other',
 'cargo trai': 'commercial',
 'cargo van': 'commercial',
 'carry all': 'commercial',
 'cat caterp': 'other',
 'cat escaba': 'other',
 'cater': 'other',
 'charter bu': 'other'

In [ ]:
# Map the dictionary onto the DataFrame columns to create new columns
df['vehicle_1_category'] = df['vehicle_type_code1'].map(type_to_cat)
df['vehicle_2_category'] = df['vehicle_type_code2'].map(type_to_cat)
df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,on_street_name,borough,zip_code,injuries_count,deaths_count,injuries,deaths,vehicle_1_category,vehicle_2_category
0,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.84866,sedan,passing or lane usage improper,sedan,unsafe lane changing,grand central pkwy,queens,11368,1,0,True,False,personal vehicles,personal vehicles
1,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.96038,sedan,unspecified,station wagon/sport utility vehicle,unspecified,henry hudson parkway,manhattan,10027,0,0,False,False,personal vehicles,personal vehicles
2,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.91799,sedan,unspecified,sedan,unspecified,saint ann's avenue,bronx,10454,0,0,False,False,personal vehicles,personal vehicles
3,2022-01-01 02:53:00,2022-01-01,02:53:00,4491586,40.646034,-73.99678,station wagon/sport utility vehicle,unspecified,station wagon/sport utility vehicle,unspecified,40th street,brooklyn,11232,0,0,False,False,personal vehicles,personal vehicles
4,2022-01-01 17:00:00,2022-01-01,17:00:00,4491660,40.701195,-73.91409,sedan,passing too closely,sedan,unspecified,wyckoff avenue,brooklyn,11237,0,0,False,False,personal vehicles,personal vehicles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54198,2022-12-31 19:48:00,2022-12-31,19:48:00,4596147,40.749786,-73.98778,motorscooter,driver inattention/distraction,taxi,unspecified,avenue of the americas,manhattan,10001,1,0,True,False,bikes & scooters,personal vehicles
54199,2022-12-31 20:45:00,2022-12-31,20:45:00,4596082,40.679680,-73.97826,station wagon/sport utility vehicle,passing or lane usage improper,e-scooter,unspecified,5 avenue,brooklyn,11217,1,0,True,False,personal vehicles,bikes & scooters
54200,2022-12-31 04:00:00,2022-12-31,04:00:00,4595952,40.739925,-73.90454,sedan,unspecified,sedan,unspecified,60 street,queens,11377,0,0,False,False,personal vehicles,personal vehicles
54201,2022-12-31 19:00:00,2022-12-31,19:00:00,4596246,40.683132,-73.88259,sedan,failure to yield right-of-way,station wagon/sport utility vehicle,unspecified,ridgewood avenue,brooklyn,11208,1,0,True,False,personal vehicles,personal vehicles


we can save the category maps to a csv so that it can later be used in a funcition to map the original ds

In [27]:
type_to_cat_df = pd.DataFrame(list(type_to_cat.items()), columns=['vehicle_types', 'category'])

type_to_cat_df.to_csv('type_to_cat_df.csv', index=False)

In [ ]:
# Load the CSV file back into a DataFrame
loaded_type_to_cat_df = pd.read_csv('type_to_cat_df.csv')

# Convert the DataFrame back to a dictionary
loaded_type_to_cat = dict(zip(loaded_type_to_cat_df['vehicle_types'], loaded_type_to_cat_df['category']))

# Map the loaded dictionary onto the DataFrame columns to create new columns
df['vehicle_1_category'] = df['vehicle_type_code1'].map(loaded_type_to_cat)
df['vehicle_2_category'] = df['vehicle_type_code2'].map(loaded_type_to_cat)
df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,on_street_name,borough,zip_code,injuries_count,deaths_count,injuries,deaths,vehicle_1_category,vehicle_2_category
0,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.84866,sedan,passing or lane usage improper,sedan,unsafe lane changing,grand central pkwy,queens,11368,1,0,True,False,personal vehicles,personal vehicles
1,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.96038,sedan,unspecified,station wagon/sport utility vehicle,unspecified,henry hudson parkway,manhattan,10027,0,0,False,False,personal vehicles,personal vehicles
2,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.91799,sedan,unspecified,sedan,unspecified,saint ann's avenue,bronx,10454,0,0,False,False,personal vehicles,personal vehicles
3,2022-01-01 02:53:00,2022-01-01,02:53:00,4491586,40.646034,-73.99678,station wagon/sport utility vehicle,unspecified,station wagon/sport utility vehicle,unspecified,40th street,brooklyn,11232,0,0,False,False,personal vehicles,personal vehicles
4,2022-01-01 17:00:00,2022-01-01,17:00:00,4491660,40.701195,-73.91409,sedan,passing too closely,sedan,unspecified,wyckoff avenue,brooklyn,11237,0,0,False,False,personal vehicles,personal vehicles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54198,2022-12-31 19:48:00,2022-12-31,19:48:00,4596147,40.749786,-73.98778,motorscooter,driver inattention/distraction,taxi,unspecified,avenue of the americas,manhattan,10001,1,0,True,False,bikes & scooters,personal vehicles
54199,2022-12-31 20:45:00,2022-12-31,20:45:00,4596082,40.679680,-73.97826,station wagon/sport utility vehicle,passing or lane usage improper,e-scooter,unspecified,5 avenue,brooklyn,11217,1,0,True,False,personal vehicles,bikes & scooters
54200,2022-12-31 04:00:00,2022-12-31,04:00:00,4595952,40.739925,-73.90454,sedan,unspecified,sedan,unspecified,60 street,queens,11377,0,0,False,False,personal vehicles,personal vehicles
54201,2022-12-31 19:00:00,2022-12-31,19:00:00,4596246,40.683132,-73.88259,sedan,failure to yield right-of-way,station wagon/sport utility vehicle,unspecified,ridgewood avenue,brooklyn,11208,1,0,True,False,personal vehicles,personal vehicles


In [29]:
vehicle_df.category.unique()

array(['commercial', 'personal', 'bikes & scooters', 'other'],
      dtype=object)

In [30]:
# Concatenate all values from 'vehicle_type_code1' and 'vehicle_type_code2' columns
all_factors = list(df['contributing_factor_vehicle_1'].values) + list(df['contributing_factor_vehicle_2'].values)

# Print the list of all vehicle types
all_vehicle_types= set(all_vehicle_types)
all_vehicle_types


{'100th truc',
 '16m',
 '2 dr sedan',
 '3 wheel sc',
 '3-door',
 '4 dr sedan',
 '50cc mini',
 '7',
 '997',
 '999',
 'a,n',
 'abulance',
 'access a r',
 'amb',
 'ambu',
 'ambulance',
 'ambulence',
 'ambulette',
 'arcimoto',
 'armored truck',
 'asphalt ro',
 'b55 excava',
 'backhoe',
 'backhoe lo',
 'beverage truck',
 'bike',
 'bobcat',
 'bobcat sno',
 'boom lift',
 'box truck',
 'boxtruck',
 'bulk agriculture',
 'bull dozer',
 'bulldozer',
 'bus',
 'cargo trai',
 'cargo van',
 'carry all',
 'cat caterp',
 'cat escaba',
 'cater',
 'charter bu',
 'chassis &',
 'chassis cab',
 'cherry pic',
 'chevrolet',
 'citibike',
 'city owned',
 'com',
 'combinatio',
 'comm truck',
 'commercial',
 'commerical',
 'con ed tru',
 'concrete mixer',
 'constructi',
 'contructio',
 'convertible',
 'crane',
 'dart bike',
 'delivery',
 'delivery t',
 'delivery v',
 'delv',
 'department',
 'detached t',
 'dirt bike',
 'dirtbike',
 'distributo',
 'dsny sanit',
 'dsny sweep',
 'dsny truck',
 'dump',
 'dump gr',
 '

In [31]:
factors_df = pd.DataFrame({'factors': list(all_factors)})
factors_df.head(2)

,factors
0,passing or lane usage improper
1,unspecified


In [32]:
factors_cat = {
    'accelerator defective': 'non_driver_related',
    'aggressive driving/road rage': 'driver_related',
    'alcohol involvement': 'driver_related',
    'animals action': 'non_driver_related',
    'backing unsafely': 'driver_related',
    'brakes defective': 'non_driver_related',
    'cell phone (hand-held)': 'driver_related',
    'cell phone (hands-free)': 'driver_related',
    'driver inattention/distraction': 'driver_related',
    'driver inexperience': 'driver_related',
    'driverless/runaway vehicle': 'driver_related',
    'drugs (illegal)': 'driver_related',
    'eating or drinking': 'driver_related',
    'failure to keep right': 'driver_related',
    'failure to yield right-of-way': 'driver_related',
    'fatigued/drowsy': 'driver_related',
    'fell asleep': 'driver_related',
    'following too closely': 'driver_related',
    'glare': 'non_driver_related',
    'headlights defective': 'non_driver_related',
    'illness': 'non_driver_related',
    'illnes': 'non_driver_related',
    'lane marking improper/inadequate': 'non_driver_related',
    'listening/using headphones': 'driver_related',
    'lost consciousness': 'driver_related',
    'obstruction/debris': 'non_driver_related',
    'other electronic device': 'driver_related',
    'other lighting defects': 'non_driver_related',
    'other vehicular': 'non_driver_related',
    'outside car distraction': 'driver_related',
    'oversized vehicle': 'non_driver_related',
    'passenger distraction': 'driver_related',
    'passing or lane usage improper': 'driver_related',
    'passing too closely': 'driver_related',
    'pavement defective': 'non_driver_related',
    'pavement slippery': 'non_driver_related',
    'pedestrian/bicyclist/other pedestrian error/confusion': 'non_driver_related',
    'physical disability': 'non_driver_related',
    'prescription medication': 'non_driver_related',
    'reaction to uninvolved vehicle': 'driver_related',
    'shoulders defective/improper': 'non_driver_related',
    'steering failure': 'non_driver_related',
    'texting': 'driver_related',
    'tinted windows': 'non_driver_related',
    'tire failure/inadequate': 'non_driver_related',
    'tow hitch defective': 'non_driver_related',
    'traffic control device improper/non-working': 'non_driver_related',
    'traffic control disregarded': 'non_driver_related',
    'turning improperly': 'driver_related',
    'unsafe lane changing': 'driver_related',
    'unsafe speed': 'driver_related',
    'unspecified': 'unspecified',
    'using on board navigation device': 'driver_related',
    'vehicle vandalism': 'non_driver_related',
    'view obstructed/limited': 'non_driver_related',
    'windshield inadequate': 'non_driver_related'
}


In [33]:
# Create a new 'description' column by mapping values from 'Vehicle Names' column
factors_df['factors_category'] = factors_df['factors'].map(factors_cat)

In [34]:
factors_sub = {
    'accelerator defective': 'Mechanical',
    'aggressive driving/road rage': 'Distraction',
    'alcohol involvement': 'Substance Abuse',
    'animals action': 'Environmental',
    'backing unsafely': 'Driving Violations',
    'brakes defective': 'Mechanical',
    'cell phone (hand-held)': 'Distraction',
    'cell phone (hands-free)': 'Distraction',
    'driver inattention/distraction': 'Distraction',
    'driver inexperience': 'Distraction',
    'driverless/runaway vehicle': 'Distraction',
    'drugs (illegal)': 'Substance Abuse',
    'eating or drinking': 'Distraction',
    'failure to keep right': 'Driving Violations',
    'failure to yield right-of-way': 'Driving Violations',
    'fatigued/drowsy': 'Health-Related',
    'fell asleep': 'Health-Related',
    'following too closely': 'Driving Violations',
    'glare': 'Environmental',
    'headlights defective': 'Mechanical',
    'illnes': 'Health-Related',
    'lane marking improper/inadequate': 'Environmental',
    'listening/using headphones': 'Distraction',
    'lost consciousness': 'Health-Related',
    'obstruction/debris': 'Environmental',
    'other electronic device': 'Distraction',
    'other lighting defects': 'Mechanical',
    'other vehicular': 'Mechanical',
    'outside car distraction': 'Distraction',
    'oversized vehicle': 'Environmental',
    'passenger distraction': 'Distraction',
    'passing or lane usage improper': 'Driving Violations',
    'passing too closely': 'Driving Violations',
    'pavement defective': 'Environmental',
    'pavement slippery': 'Environmental',
    'pedestrian/bicyclist/other pedestrian error/confusion': 'Environmental',
    'physical disability': 'Health-Related',
    'prescription medication': 'Substance Abuse',
    'reaction to uninvolved vehicle': 'Distraction',
    'shoulders defective/improper': 'Mechanical',
    'steering failure': 'Mechanical',
    'texting': 'Distraction',
    'tinted windows': 'Mechanical',
    'tire failure/inadequate': 'Mechanical',
    'tow hitch defective': 'Mechanical',
    'traffic control device improper/non-working': 'Environmental',
    'traffic control disregarded': 'Environmental',
    'turning improperly': 'Driving Violations',
    'unsafe lane changing': 'Driving Violations',
    'unsafe speed': 'Driving Violations',
    'unspecified': 'unspecified',
    'using on board navigation device': 'Distraction',
    'vehicle vandalism': 'Environmental',
    'view obstructed/limited': 'Environmental',
    'windshield inadequate': 'Mechanical'
}

In [35]:
# Create a new 'description' column by mapping values from 'Vehicle Names' column
factors_df['factors_subcat'] = factors_df['factors'].map(factors_sub)

In [36]:
factors_df['factors'] = factors_df['factors'].str.lower()
factors_df['factors_category'] = factors_df['factors_category'].str.lower()
factors_df['factors_subcat'] = factors_df['factors_subcat'].str.lower()
factors_df.head(3)


,factors,factors_category,factors_subcat
0,passing or lane usage improper,driver_related,driving violations
1,unspecified,unspecified,unspecified
2,unspecified,unspecified,unspecified


In [37]:
# Create a dictionary mapping vehicle_types to category
factors_to_cat = dict(zip(factors_df['factors'], factors_df['factors_category']))
factors_to_cat

{'passing or lane usage improper': 'driver_related',
 'unspecified': 'unspecified',
 'passing too closely': 'driver_related',
 'driver inattention/distraction': 'driver_related',
 'traffic control disregarded': 'non_driver_related',
 'backing unsafely': 'driver_related',
 'brakes defective': 'non_driver_related',
 'alcohol involvement': 'driver_related',
 'unsafe lane changing': 'driver_related',
 'following too closely': 'driver_related',
 'pedestrian/bicyclist/other pedestrian error/confusion': 'non_driver_related',
 'fell asleep': 'driver_related',
 'unsafe speed': 'driver_related',
 'reaction to uninvolved vehicle': 'driver_related',
 'view obstructed/limited': 'non_driver_related',
 'pavement slippery': 'non_driver_related',
 'turning improperly': 'driver_related',
 'tire failure/inadequate': 'non_driver_related',
 'failure to yield right-of-way': 'driver_related',
 'driver inexperience': 'driver_related',
 'aggressive driving/road rage': 'driver_related',
 'lane marking improper/

In [38]:
# Create a dictionary mapping vehicle_types to category
factors_to_subcat = dict(zip(factors_df['factors'], factors_df['factors_subcat']))
factors_to_subcat

{'passing or lane usage improper': 'driving violations',
 'unspecified': 'unspecified',
 'passing too closely': 'driving violations',
 'driver inattention/distraction': 'distraction',
 'traffic control disregarded': 'environmental',
 'backing unsafely': 'driving violations',
 'brakes defective': 'mechanical',
 'alcohol involvement': 'substance abuse',
 'unsafe lane changing': 'driving violations',
 'following too closely': 'driving violations',
 'pedestrian/bicyclist/other pedestrian error/confusion': 'environmental',
 'fell asleep': 'health-related',
 'unsafe speed': 'driving violations',
 'reaction to uninvolved vehicle': 'distraction',
 'view obstructed/limited': 'environmental',
 'pavement slippery': 'environmental',
 'turning improperly': 'driving violations',
 'tire failure/inadequate': 'mechanical',
 'failure to yield right-of-way': 'driving violations',
 'driver inexperience': 'distraction',
 'aggressive driving/road rage': 'distraction',
 'lane marking improper/inadequate': 'e

In [40]:
# Map the dictionary onto the DataFrame columns to create new columns
df['factors_category_vehicle_1'] = df['contributing_factor_vehicle_1'].map(factors_to_cat)
df['factors_category_vehicle_2'] = df['contributing_factor_vehicle_2'].map(factors_to_cat)
df['factors_subcat_vehicle_1'] = df['contributing_factor_vehicle_1'].map(factors_to_subcat)
df['factors_subcat_vehicle_2'] = df['contributing_factor_vehicle_2'].map(factors_to_subcat)

df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,...,injuries_count,deaths_count,injuries,deaths,vehicle_1_category,vehicle_2_category,factors_category_vehicle_1,factors_category_vehicle_2,factors_subcat_vehicle_1,factors_subcat_vehicle_2
0,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.84866,sedan,passing or lane usage improper,sedan,unsafe lane changing,...,1,0,True,False,personal vehicles,personal vehicles,driver_related,driver_related,driving violations,driving violations
1,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.96038,sedan,unspecified,station wagon/sport utility vehicle,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,non_driver_related,non_driver_related,environmental,environmental
2,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.91799,sedan,unspecified,sedan,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,non_driver_related,non_driver_related,environmental,environmental
3,2022-01-01 02:53:00,2022-01-01,02:53:00,4491586,40.646034,-73.99678,station wagon/sport utility vehicle,unspecified,station wagon/sport utility vehicle,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,non_driver_related,non_driver_related,environmental,environmental
4,2022-01-01 17:00:00,2022-01-01,17:00:00,4491660,40.701195,-73.91409,sedan,passing too closely,sedan,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,driver_related,non_driver_related,driving violations,environmental
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54198,2022-12-31 19:48:00,2022-12-31,19:48:00,4596147,40.749786,-73.98778,motorscooter,driver inattention/distraction,taxi,unspecified,...,1,0,True,False,bikes & scooters,personal vehicles,driver_related,non_driver_related,distraction,environmental
54199,2022-12-31 20:45:00,2022-12-31,20:45:00,4596082,40.679680,-73.97826,station wagon/sport utility vehicle,passing or lane usage improper,e-scooter,unspecified,...,1,0,True,False,personal vehicles,bikes & scooters,driver_related,non_driver_related,driving violations,environmental
54200,2022-12-31 04:00:00,2022-12-31,04:00:00,4595952,40.739925,-73.90454,sedan,unspecified,sedan,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,non_driver_related,non_driver_related,environmental,environmental
54201,2022-12-31 19:00:00,2022-12-31,19:00:00,4596246,40.683132,-73.88259,sedan,failure to yield right-of-way,station wagon/sport utility vehicle,unspecified,...,1,0,True,False,personal vehicles,personal vehicles,driver_related,non_driver_related,driving violations,environmental


we can save the category maps to a csv so that it can later be used in a funcition to map the original ds

In [39]:
# type_to_cat_df = pd.DataFrame(list(type_to_cat.items()), columns=['vehicle_types', 'category'])

factors_df.to_csv('factors_df.csv', index=False)

In [42]:
# Load the CSV file back into a DataFrame
factors_df_loaded = pd.read_csv('factors_df.csv')

# Convert the DataFrame back to a dictionary
factors_to_cat = dict(zip(factors_df_loaded['factors'], factors_df_loaded['factors_category']))

# Convert the DataFrame back to a dictionary
factors_to_subcat = dict(zip(factors_df_loaded['factors'], factors_df_loaded['factors_subcat']))

# Map the dictionary onto the DataFrame columns to create new columns
df['factors_category_vehicle_1'] = df['contributing_factor_vehicle_1'].map(factors_to_cat)
df['factors_category_vehicle_2'] = df['contributing_factor_vehicle_2'].map(factors_to_cat)
df['factors_subcat_vehicle_1'] = df['contributing_factor_vehicle_1'].map(factors_to_subcat)
df['factors_subcat_vehicle_2'] = df['contributing_factor_vehicle_2'].map(factors_to_subcat)

df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,...,injuries_count,deaths_count,injuries,deaths,vehicle_1_category,vehicle_2_category,factors_category_vehicle_1,factors_category_vehicle_2,factors_subcat_vehicle_1,factors_subcat_vehicle_2
0,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.84866,sedan,passing or lane usage improper,sedan,unsafe lane changing,...,1,0,True,False,personal vehicles,personal vehicles,driver_related,driver_related,driving violations,driving violations
1,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.96038,sedan,unspecified,station wagon/sport utility vehicle,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,non_driver_related,non_driver_related,environmental,environmental
2,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.91799,sedan,unspecified,sedan,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,non_driver_related,non_driver_related,environmental,environmental
3,2022-01-01 02:53:00,2022-01-01,02:53:00,4491586,40.646034,-73.99678,station wagon/sport utility vehicle,unspecified,station wagon/sport utility vehicle,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,non_driver_related,non_driver_related,environmental,environmental
4,2022-01-01 17:00:00,2022-01-01,17:00:00,4491660,40.701195,-73.91409,sedan,passing too closely,sedan,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,driver_related,non_driver_related,driving violations,environmental
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54198,2022-12-31 19:48:00,2022-12-31,19:48:00,4596147,40.749786,-73.98778,motorscooter,driver inattention/distraction,taxi,unspecified,...,1,0,True,False,bikes & scooters,personal vehicles,driver_related,non_driver_related,distraction,environmental
54199,2022-12-31 20:45:00,2022-12-31,20:45:00,4596082,40.679680,-73.97826,station wagon/sport utility vehicle,passing or lane usage improper,e-scooter,unspecified,...,1,0,True,False,personal vehicles,bikes & scooters,driver_related,non_driver_related,driving violations,environmental
54200,2022-12-31 04:00:00,2022-12-31,04:00:00,4595952,40.739925,-73.90454,sedan,unspecified,sedan,unspecified,...,0,0,False,False,personal vehicles,personal vehicles,non_driver_related,non_driver_related,environmental,environmental
54201,2022-12-31 19:00:00,2022-12-31,19:00:00,4596246,40.683132,-73.88259,sedan,failure to yield right-of-way,station wagon/sport utility vehicle,unspecified,...,1,0,True,False,personal vehicles,personal vehicles,driver_related,non_driver_related,driving violations,environmental


In [10]:
# Example usage:
info_dataframe = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
info_dataframe

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,41429,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1440,object
3,collision_id,0,0.0,0,0,48254,int64
4,latitude,0,0.0,0,0,23107,float64
5,longitude,0,0.0,0,0,19572,float64
6,on_street_name,0,0.0,0,0,4518,object
7,borough,1,0.0,0,0,6,object
8,zip_code,0,0.0,0,0,211,int64
9,injuries_count,0,0.0,32376,0,14,int64


In [9]:
df = df.drop(columns=['vehicle_type_code1', 'contributing_factor_vehicle_1', 'vehicle_type_code2', 'contributing_factor_vehicle_2'])

In [53]:
# df = pd.read_csv('explore_df_pre_datetime.csv', index_col=0)

In [20]:
# Time of Day
bins = [0, 6, 12, 18, 24]
labels = ['night', 'morning', 'afternoon', 'evening']
df['time_of_day'] = pd.cut(df['crash_datetime'].dt.hour, bins=bins, labels=labels, right=False, include_lowest=True)

# Hour
df['hour'] = df['crash_datetime'].dt.hour


# Updating the features as per the request

# Day of the week using day names
df['day_of_week'] = df['crash_datetime'].dt.day_name()

# Month using month names
df['month'] = df['crash_datetime'].dt.month_name()

# Is Weekend (using boolean values)
df['is_weekend'] = df['crash_datetime'].dt.dayofweek.isin([5, 6])

# Is Daylight (using boolean values)
df['is_daylight'] = (df['hour'] >= 6) & (df['hour'] <= 19)

df[['crash_datetime', 'day_of_week', 'time_of_day', 'month', 'hour', 'is_weekend', 'is_daylight']].head()

,crash_datetime,day_of_week,time_of_day,month,hour,is_weekend,is_daylight
0,2022-01-01 05:17:00,Saturday,night,January,5,True,False
1,2022-01-01 01:30:00,Saturday,night,January,1,True,False
2,2022-01-01 16:40:00,Saturday,afternoon,January,16,True,True
3,2022-01-01 02:53:00,Saturday,night,January,2,True,False
4,2022-01-01 17:00:00,Saturday,afternoon,January,17,True,True


In [23]:
# Create a new feature for hours between midnight and 4 am on Saturday and Sunday
df['late_weekend'] = ((df['hour'] >= 0) & (df['hour'] <= 4) & df['is_weekend'])

# Display the resulting DataFrame
df[['crash_datetime', 'day_of_week', 'time_of_day', 'month', 'hour', 'is_weekend', 'is_daylight', 'late_weekend']]


,crash_datetime,day_of_week,time_of_day,month,hour,is_weekend,is_daylight,late_weekend
0,2022-01-01 05:17:00,Saturday,night,January,5,True,False,False
1,2022-01-01 01:30:00,Saturday,night,January,1,True,False,True
2,2022-01-01 16:40:00,Saturday,afternoon,January,16,True,True,False
3,2022-01-01 02:53:00,Saturday,night,January,2,True,False,True
4,2022-01-01 17:00:00,Saturday,afternoon,January,17,True,True,False
...,...,...,...,...,...,...,...,...
54198,2022-12-31 19:48:00,Saturday,evening,December,19,True,True,False
54199,2022-12-31 20:45:00,Saturday,evening,December,20,True,False,False
54200,2022-12-31 04:00:00,Saturday,night,December,4,True,False,True
54201,2022-12-31 19:00:00,Saturday,evening,December,19,True,True,False


In [ ]:
# Assuming you have a DataFrame named df
df['borough'] = df['borough'].replace('the bronx', 'bronx')

In [29]:
# Save the DataFrame to a CSV file
df.to_csv('explore_df.csv')

In [27]:
# Save the DataFrame to a CSV file
df.head(200).to_csv('df200.csv', index=False)